## Importing Libraries  
First we’ll need to import some libraries and a number from the LightFM library.

In [ ]:
import numpy as np
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.cross_validation import random_train_test_split
from scipy.sparse import coo_matrix as sp

## Prepare the Data  
To train the model using LightFM we will supply the following:  
 1. interactions as a sparse.coo_matrix – our interactions will be UseGuiD and ProductGuid
 2. user_features as an iterable of strings containing user features – Company Size,Industry Type, location, etc
 3. item_features as an iterable of strings containing item features – Product Category, Color, etc
 
LightFM has a dataset constructor with a number of handy methods to get our data ready to input into the model. As we want to include user and item features in our model, preparing the data will be a two step process.

### Step 1: Create the feature mappings
First, we use the fit() method on the dataset object to create the mappings.  
Second, can call the build_interactions() method on the dataset object to build the interactions matrix.

In [ ]:
subject = 'user/customer'
dataset = Dataset()
df_interaction = order_dframe_model.copy()
arr2D_interaction = df_interaction[[subject,'ProductGuid','ratings']].to_numpy()
del df_interaction
dataset.fit(users=df_interaction[subject].unique(), items=df_interaction.ProductId.unique())
(interactions, weights) = dataset.build_interactions((tuple(row) for row in arr2D_interaction))
num_users, num_items = dataset.interactions_shape()
print(‘Num users: {}, num_items {}.’.format(num_users, num_items))

We use the dataset.fit_partial() method to create the feature mappings for user_features and model_features.

In [ ]:
arr2D_userFeature = feature_user.to_numpy()
arr2D_itemFeature = feature_item.to_numpy()
del feature_user
del feature_item
dataset.fit_partial(users=(row[‘UserGuid’] for row in arr2D_userFeature), 
                    items=(row[‘ProductGuid’] for row in arr2D_itemFeature), 
                    item_features=(row[‘ParentColectionGuid’] for row in arr2D_itemFeature), 
                    user_features=(row[‘Location’] for row in arr2D_userFeature))

## Step 2: Build the interactions, user feature and item feature matrices:  
We call the build_item_features() and build_user_features() methods on the dataset object to build the item_features and user_features. These return objects of type sparse.coo_matrix as required by LightFM.

In [ ]:
item_features = dataset.build_item_features(((row[‘ProductGuid’], [row[‘ParentCategory’]]) for row in arr2D_itemFeature))
user_features = dataset.build_user_features(((row[‘UserGuid’], [row[‘location’]]) for row in arr2D_userFeature))

## Specifying the Model  

To train the recommender we will use the Weighted Approximate Rank Pair-Wise (WARP) loss function provided in the LightFM library. WARP deals with (user, positive item, negative item) triplets.
LightFM provides the following explanation of the WARP algorithm:
1. For a given (user, positive item pair), sample a negative item at random from all the remaining items. Compute predictions for both items; if the negative item’s prediction exceeds that of the positive item plus a margin, perform a gradient update to rank the positive item higher and the negative item lower. If there is no rank violation, continue sampling negative items until a violation is found.
2. If you found a violating negative example at the first try, make a large gradient update: this indicates that a lot of negative items are ranked higher than positives items given the current state of the model, and the model must be updated by a large amount. If it took a lot of sampling to find a violating example, perform a small update: the model is likely close to the optimum and should be updated at a low rate.  

Here’s how we specify the model with the WARP loss function:

In [ ]:
 model = LightFM(loss=’warp’)

The model can be specified with numerous hyperparameters by importing lightFM_bestParameter.ipnb file. It is worth mentioning that LightFM also allows Bayesian Personalised Ranking loss, though this routinely performs less well.  

## Training the Model  

We can then call .fit() to fit the model to the interactions, the item and user feature sets.

In [ ]:
model.fit(interactions, item_features=item_features, user_features=user_features, epochs=2, sample_weight=weights)

### Once, model is save and retrieve then we can use fit_partial considering only rest of data 

In [ ]:
saved_model.fit_partial(partial_interactions, etc)